In [1]:
#Does not work on Windows, probably
import multiprocessing
import os
import shutil
import subprocess
import tempfile  # cannot use

import pexpect  # requires github pexpect

In [2]:
my_app = '../../broken_app/broken_app.py'

## Here we define the wrapper

In [3]:
def run_batch(pipe, temp_dir, app, op):
    os.chdir(temp_dir)
    #This is enough to run simple interactive processes
    proc = subprocess.Popen(['../' + app], bufsize=1, universal_newlines=True,
                            stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = proc.communicate('%d\n' % op)
    pipe.send(out.split('\n')[-1].rstrip())

def run_interactive(pipe, temp_dir, app, op):
    os.chdir(temp_dir)
    c = pexpect.spawnu('../' + app)
    c.expect(u'\?')
    #Explain the problem with popen
    c.sendline(u'%d' % op)
    pipe.send(c.readlines()[-1].rstrip())
    os.chdir('..')  # not necessary


def run_app(app, op, in_file, out_file, interactive=False):
    parent, child = multiprocessing.Pipe()
    temp_dir = tempfile.mkdtemp(dir='.')  # Dir should be on the local FS
    os.symlink('../%s' % in_file, '%s/in.params' % temp_dir)  # Link, not copy - could be gigantic file
    if not interactive:
        p = multiprocessing.Process(target=run_batch, args=(child, temp_dir, app, op))
    else:
        p = multiprocessing.Process(target=run_interactive, args=(child, temp_dir, app, op))
    p.start()
    line = parent.recv()
    p.join()
    shutil.rmtree(temp_dir)
    #No command-line arguments
    #instead of call
    #Pipe


## Using the wrapper

In [4]:
def create_dummy_input(fname, n1, n2):
    w = open(fname, 'w')
    w.write('%f\n%f\n' % (n1, n2))
    w.close()

create_dummy_input('my.params', 1.5, 2)
op = 1  # subtraction
run_app(my_app, op, 'my.params', 'out.result', interactive=True)

In [5]:
os.listdir('../../broken_app')

['in.params', 'broken_app.py']